# Data wrangling pipeline

## Setup

In [2]:
# Imports
import polars as pl
from datetime import datetime as dt
import logging

# Initialise variables
pp_cols = ['id', 'price', 'date', 'postcode', 'property_type', 'newly_built', 'freehold_or_lease', 'paon', 'soan', 'street', 'locality', 'town_city', 'district', 'county']
pc_cols = ['postcode', 'latitude', 'longitude']

In [9]:
pp_df = pl.scan_parquet('manchester_property_sales\\data\\pp_data_man.parquet')
pp_df = pp_df.rename(dict(zip(pp_df.collect_schema().names(), pp_cols)))

pc_df = pl.scan_parquet('manchester_property_sales\\data\\pc_man.parquet')
pc_df = pc_df.rename(dict(zip(pc_df.collect_schema().names(), pc_cols)))

## Cleaning and transforming methods

In [10]:
def validate_re(df: pl.LazyFrame, re: str, col: str) -> None:
    ''' Validates whether a column conforms to format specified by a regular expression'''
    invalid = df.filter(~pl.col(col).str.contains(re))
    num_invalid = invalid.select(pl.len()).collect().item()

    if num_invalid > 0:
        raise Exception(f"{num_invalid} {col} values are invalid")

In [11]:
postcode_re = r'^[A-Z0-9]{2,4} [A-Z0-9]{3}$'

def clean_transform_postcode(postcode_re: str) -> list[pl.Expr]:
    # remove leading and trailing spaces
    # replace any internal spaces with a single space
    # convert letters to uppercase
    cleaned_postcode = (
        pl.col('postcode')
        .str.strip_chars(' ')
        .str.replace_all(r'\s+', ' ')
        .str.to_uppercase()
        .alias('postcode')
    )
    
    invalid_postcode = ~cleaned_postcode.str.contains(postcode_re)
    return [
        pl.when(invalid_postcode)
        .then(None)
        .otherwise(pl.col('postcode'))
        .alias('postcode'),
        
        pl.when(invalid_postcode)
        .then(None)
        .otherwise(pl.col('postcode').str.extract(r'^(\S+ \d)'))
        .alias('postal_sector')
    ]

pp_df.with_columns(clean_transform_postcode(postcode_re)).filter(pl.col('postcode').is_null()).collect()

id,price,date,postcode,property_type,newly_built,freehold_or_lease,paon,soan,street,locality,town_city,district,county,postal_sector
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""{BEFB0DDC-5B5E-4316-B958-9720A…","""485,000.00""","""2008-04-04 00:00""",null,"""D""","""N""","""F""","""LAND ADJOINING, 877 - 879""","""""","""MOSS BANK WAY""","""BOLTON""","""BOLTON""","""BOLTON""","""GREATER MANCHESTER""",null
"""{42A5A70A-20E5-56E8-E050-A8C06…","""125,000.00""","""2015-12-18 00:00""",null,"""O""","""N""","""L""","""PARKING SPACE S160""","""""","""ALDER STREET""","""""","""SALFORD""","""SALFORD""","""GREATER MANCHESTER""",null
"""{31C0FEEB-9156-4C6C-B2FE-85CC7…","""36,500.00""","""2003-12-04 00:00""",null,"""D""","""N""","""F""","""THE LODGE""","""""","""ROMAN ROAD""","""OLDHAM""","""OLDHAM""","""OLDHAM""","""GREATER MANCHESTER""",null
"""{E7B085FD-2F7C-7E31-E053-6C04A…","""410,000.00""","""2022-03-29 00:00""",null,"""O""","""N""","""F""","""148""","""""","""MOSLEY COMMON ROAD""","""TYLDESLEY""","""MANCHESTER""","""WIGAN""","""GREATER MANCHESTER""",null
"""{21E5FEB7-69BB-2439-E050-A8C06…","""800,000.00""","""2015-06-08 00:00""",null,"""O""","""N""","""F""","""UNIT 5 BUFFOLINE TRADING ESTAT…","""""","""CHAPEL STREET""","""LEVENSHULME""","""MANCHESTER""","""MANCHESTER""","""GREATER MANCHESTER""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""{582D0637-EF41-8F22-E053-6C04A…","""2,500.00""","""2017-06-22 00:00""",null,"""O""","""N""","""L""","""LAWNHURST, 826""","""STORAGE ROOM 2""","""WILMSLOW ROAD""","""""","""MANCHESTER""","""MANCHESTER""","""GREATER MANCHESTER""",null
"""{2D4D7609-E200-BDF9-E063-4804A…","""15,000.00""","""2023-10-09 00:00""",null,"""O""","""N""","""F""","""LAND ADJOINING, 59""","""""","""GREAT MOOR STREET""","""""","""STOCKPORT""","""STOCKPORT""","""GREATER MANCHESTER""",null
"""{AE4D86D4-ACEE-4619-E053-6C04A…","""7,800.00""","""2019-10-16 00:00""",null,"""O""","""N""","""F""","""PLOTS 36 AND 37""","""""","""BURY ROAD""","""""","""BOLTON""","""BOLTON""","""GREATER MANCHESTER""",null


In [12]:
def clean_long_lat(df: pl.LazyFrame) -> pl.LazyFrame:
    return (
        df.filter(
            (pl.col('latitude') >= 49) & (pl.col('latitude') <= 61) &
            (pl.col('longitude') >= -8) & (pl.col('longitude') <= 2)
        )
    )

def validate_long_lat(df: pl.LazyFrame) -> bool:
    ''' Validates whether a longitude and latitude columns are within the UK'''
    # Check longitudes and latitude are within the UK
    valid_lon_lat = (
        df.filter(
            (pl.col('latitude') >= 49) & (pl.col('latitude') <= 61) &
            (pl.col('longitude') >= -8) & (pl.col('longitude') <= 2)
        )
    )
    if df.height != valid_lon_lat.height: 
        raise Exception(f'{df.height - valid_lon_lat.height} longitudes and latitude pairs are invalid')

# TODO: maybe check postcodes against long lat pairs using geopy or another geolocation module

In [13]:
price_re = r'^\d+(\.\d+)?$'

def clean_transform_price(price_re: str) -> pl.Expr:
    '''Cleans price column by remove anything that's not 0-9 or .'''
    cleaned_price = (
        pl.col('price')
        .str.replace_all(r'[^0-9\.]', '')           # remove anything that's not 0-9 or .
        .alias('price')
    )
    
    '''Converts prices to float if in correct format otherwise replaces it with null'''
    invalid_price = ~cleaned_price.str.contains(price_re)
    parsed = cleaned_price.cast(pl.Float64)
    cleaned_price = (
        pl.when(invalid_price)
        .then(None)
        .otherwise(pl.col('price').cast(pl.Float64))
    )

    return pl.when(invalid_price | (~parsed.is_between(10_000, 1_000_000_000))) \
    .then(None) \
    .otherwise(parsed) \
    .alias('price')

pp_df.with_columns(clean_transform_price(price_re)).filter(pl.col('price').ge(1_000_000_00)).collect()

id,price,date,postcode,property_type,newly_built,freehold_or_lease,paon,soan,street,locality,town_city,district,county
str,f64,str,str,str,str,str,str,str,str,str,str,str,str
"""{55BDCAE6-D4D7-521D-E053-6B04A…",1.07086856e8,"""2017-02-17 00:00""","""M4 2BS""","""O""","""N""","""F""","""THE PRINTWORKS, 27""","""""","""WITHY GROVE""","""""","""MANCHESTER""","""MANCHESTER""","""GREATER MANCHESTER"""
"""{64342BFF-56A8-422C-E053-6C04A…",1.03033044e8,"""2018-01-05 00:00""","""M3 3EB""","""O""","""N""","""L""","""3""","""""","""HARDMAN SQUARE""","""""","""MANCHESTER""","""MANCHESTER""","""GREATER MANCHESTER"""
"""{DBA933FA-7ADF-669D-E053-6B04A…",1.06772324e8,"""2021-10-14 00:00""","""OL16 4NZ""","""O""","""N""","""F""","""ASDA ROCHDALE ASC/CDC""","""""","""JAMES KEARNS AVENUE""","""KINGSWAY BUSINESS PARK""","""ROCHDALE""","""ROCHDALE""","""GREATER MANCHESTER"""
"""{DE2D0CE0-8BBC-51EE-E053-6C04A…",1.06772324e8,"""2021-10-21 00:00""","""OL16 4NZ""","""O""","""N""","""F""","""ASDA ROCHDALE ASC/CDC""","""""","""JAMES KEARNS AVENUE""","""KINGSWAY BUSINESS PARK""","""ROCHDALE""","""ROCHDALE""","""GREATER MANCHESTER"""
"""{E53EDD2E-ABFE-83EC-E053-6B04A…",2.92e8,"""2021-12-17 00:00""","""M3 3AQ""","""O""","""N""","""L""","""1""","""""","""HARDMAN BOULEVARD""","""""","""MANCHESTER""","""MANCHESTER""","""GREATER MANCHESTER"""


In [14]:
dt_re = r'^\d{4}-\d{1,2}-\d{1,2} \d{2}:\d{2}$'

def clean_transform_date(dt_re: str) -> pl.Expr:
    '''Cleans dates'''
    # remove leading and trailing spaces
    # replace any internal spaces with a single space
    # remove anything that's not 0-9 or - or white space
    cleaned_date = (
        pl.col('date')
        .str.strip_chars(' ')
        .str.replace_all(r'\s+', ' ')
        .str.replace_all(r'[^0-9\-\:\s]', '')
    )
    
    ''' Converts dates to datetime type if in correct format otherwise replaces it with null'''
    month = cleaned_date.str.extract(r'^\d{4}-(\d{1,2})-').cast(pl.Int16)
    day = cleaned_date.str.extract(r'^\d{4}-\d{1,2}-(\d{1,2})').cast(pl.Int16)

    # invalid if not matching, or month/day missing, or out of range
    invalid_mask = (
        (~cleaned_date.str.contains(dt_re)) |
        month.is_null() |
        day.is_null() |
        (~month.is_between(1, 12)) |
        (~day.is_between(1, 31))
    )

    # parsed datetime (used only in the otherwise branch)
    parsed = cleaned_date.str.strptime(pl.Datetime, '%Y-%m-%d %H:%M')

    return pl.when(invalid_mask).then(None).otherwise(parsed).alias('date')

pp_df.with_columns(clean_transform_date(dt_re)).collect()

id,price,date,postcode,property_type,newly_built,freehold_or_lease,paon,soan,street,locality,town_city,district,county
str,str,datetime[μs],str,str,str,str,str,str,str,str,str,str,str
"""{D93B27B1-CBD4-3100-E053-6C04A…","""140,000.00""",2022-02-16 00:00:00,"""SK16 4DT""","""T""","""N""","""F""","""70""","""""","""CHAPEL STREET""","""""","""DUKINFIELD""","""TAMESIDE""","""GREATER MANCHESTER"""
"""{2ACACE8D-02B4-295E-E063-4804A…","""278,000.00""",2024-11-29 00:00:00,"""SK6 1QW""","""S""","""N""","""F""","""1""","""""","""BRIAR GROVE""","""WOODLEY""","""STOCKPORT""","""STOCKPORT""","""GREATER MANCHESTER"""
"""{01EB45EF-F1B0-40F3-E063-4704A…","""345,448.00""",2022-02-11 00:00:00,"""M1 2EY""","""F""","""Y""","""L""","""72""","""FLAT 902""","""CHAPELTOWN STREET""","""""","""MANCHESTER""","""MANCHESTER""","""GREATER MANCHESTER"""
"""{879537F9-FB6B-4663-A0E5-5E407…","""93,000.00""",2004-09-22 00:00:00,"""BL8 2RR""","""T""","""N""","""L""","""43""","""""","""NEWBOLD STREET""","""BURY""","""BURY""","""BURY""","""GREATER MANCHESTER"""
"""{7011B10A-2B91-8ED6-E053-6B04A…","""80,000.00""",2018-05-03 00:00:00,"""BL3 4HE""","""T""","""N""","""F""","""243""","""""","""WILLOWS LANE""","""""","""BOLTON""","""BOLTON""","""GREATER MANCHESTER"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""{B83BB2E2-7A4C-41E6-A67B-1CBE9…","""94,130.00""",2013-05-31 00:00:00,"""M14 4EZ""","""T""","""N""","""F""","""34""","""""","""GREAT SOUTHERN STREET""","""""","""MANCHESTER""","""MANCHESTER""","""GREATER MANCHESTER"""
"""{42A5A709-54F7-56E8-E050-A8C06…","""142,000.00""",2016-09-30 00:00:00,"""BL4 8NU""","""D""","""N""","""L""","""35""","""""","""GREENMOUNT PARK""","""KEARSLEY""","""BOLTON""","""BOLTON""","""GREATER MANCHESTER"""
"""{98C75471-F554-72E9-E053-6B04A…","""265,000.00""",2019-10-04 00:00:00,"""M16 0HU""","""T""","""N""","""L""","""70""","""""","""WARWICK ROAD SOUTH""","""""","""MANCHESTER""","""TRAFFORD""","""GREATER MANCHESTER"""


In [15]:
def clean_category(colm: str, categories: list[str]) -> pl.Expr:
    '''Replace values not in `categories` with None in the given column with categorical data'''
    return pl.when(~pl.col(colm).is_in(categories)) \
    .then(None) \
    .otherwise(pl.col(colm)) \
    .alias(colm)

test = clean_category('property_type', ['F', 'S', 'D', 'T', 'O'])
test = clean_category('newly_built', ['Y', 'N'])
test = clean_category('freehold_or_lease', ['F', 'L'])
pp_df.with_columns(test).filter(pl.col('freehold_or_lease').is_null()).collect()

id,price,date,postcode,property_type,newly_built,freehold_or_lease,paon,soan,street,locality,town_city,district,county
str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""{870C1145-73D1-4226-B820-F545B…","""139,995.00""","""2003-02-28 00:00""","""M27 8AE""","""D""","""Y""",null,"""4""","""""","""KILCOBY AVENUE""","""SWINTON""","""MANCHESTER""","""SALFORD""","""GREATER MANCHESTER"""
"""{7715847F-F778-4125-BA18-36B1D…","""98,950.00""","""2003-06-12 00:00""","""M14 7BB""","""F""","""N""",null,"""THE COTTAGES""","""FLAT 1A""","""HART ROAD""","""MANCHESTER""","""MANCHESTER""","""MANCHESTER""","""GREATER MANCHESTER"""
"""{3AC57474-E36E-44F0-BDD9-27FA0…","""40,000.00""","""1996-11-29 00:00""","""SK4 2NR""","""F""","""N""",null,"""56F""","""""","""NORRIS HILL DRIVE""","""STOCKPORT""","""STOCKPORT""","""STOCKPORT""","""GREATER MANCHESTER"""
"""{1A98CC0B-284B-4C58-A835-94287…","""180,000.00""","""2006-04-12 00:00""","""M41 9HQ""","""S""","""N""",null,"""107""","""""","""CUMBERLAND ROAD""","""URMSTON""","""MANCHESTER""","""TRAFFORD""","""GREATER MANCHESTER"""
"""{7273456B-C117-48B5-A4A0-B0EB1…","""26,000.00""","""1995-02-20 00:00""","""OL8 3DE""","""T""","""N""",null,"""300""","""""","""COPSTER HILL ROAD""","""OLDHAM""","""OLDHAM""","""OLDHAM""","""GREATER MANCHESTER"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""{11CA0F13-E968-4C20-AF0E-BCC28…","""103,000.00""","""2002-10-08 00:00""","""OL9 0PQ""","""S""","""N""",null,"""42""","""""","""CHADDERTON PARK ROAD""","""CHADDERTON""","""OLDHAM""","""OLDHAM""","""GREATER MANCHESTER"""
"""{2A289E9D-2A80-CDC8-E050-A8C06…","""93,000.00""","""2003-07-08 00:00""","""M9 6FR""","""S""","""N""",null,"""1200""","""""","""ROCHDALE ROAD""","""""","""MANCHESTER""","""MANCHESTER""","""GREATER MANCHESTER"""
"""{626881E1-55F2-433F-AECB-1B4BD…","""70,000.00""","""2001-10-05 00:00""","""M20 2WW""","""F""","""N""",null,"""73""","""FLAT 5""","""CLYDE ROAD""","""MANCHESTER""","""MANCHESTER""","""MANCHESTER""","""GREATER MANCHESTER"""


## Logging

In [4]:
def setup_logger():
    # Generate a fresh timestamp for each run
    log_filename = f"logs\\pipeline_{dt.now().strftime('%Y-%m-%d_%H-%M-%S')}.log"

    # Create handlers
    file_handler = logging.FileHandler(log_filename, mode="w")
    console_handler = logging.StreamHandler()

    # Set levels
    file_handler.setLevel(logging.INFO)
    console_handler.setLevel(logging.INFO)

    # Create formatter
    formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")

    file_handler.setFormatter(formatter)
    console_handler.setFormatter(formatter)

    # Get root logger
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    # Clear old handlers if they exist (important in Jupyter/interactive runs)
    if logger.hasHandlers():
        logger.handlers.clear()

    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    return logger

## Pipeline method

In [18]:
postcode_re = r'^[A-Z0-9]{2,4} [A-Z0-9]{3}$'
price_re = r'^\d+(\.\d+)?$'
dt_re = r'^\d{4}-\d{1,2}-\d{1,2} \d{2}:\d{2}$'

# Clean columns individually
def pipeline(
        path_to_pp_df: str = 'manchester_property_sales\\data\\pp_data_man.parquet', 
        path_to_pc_df: str = 'manchester_property_sales\\data\\pc_man.parquet',
        counties: list[str] = ['GREATER MANCHESTER'],
        start_date: dt = dt(1996, 1, 1),
        end_date: dt = dt(2024, 12, 31)
    ):
    '''Pipeline for cleaning and combining `prices paid` and `postcodes` datasets
    
    Parameters
    ----------
    path_to_pp_df: str
        Path to paid prices dataset. This can be a local file or a an online url
    path_to_pc_df: str
        Path to postcodes dataset. This can be a local file or a an online url
    counties: list[str]
        Rows with counties not in the `counties` list will be filtered out
    start_date: dt
        Rows with dates before `start_date` will be filtered out
    end_date: dt
        Rows with dates after `start_date` will be filtered out
    
    Returns
    -------
    pl.DataFrame
        A cleaned and filtered prices paid dataset with longitudes and latitudes added from the postcodes dataset
    '''
    # Set up log file
    logger = setup_logger()
    cleaned_df = None
    
    try:
        # Load data
        logger.info("Loading data...")
        pp_df = pl.scan_parquet(path_to_pp_df)
        pp_df = pp_df.rename(dict(zip(pp_df.collect_schema().names(), pp_cols)))
        pc_df = pl.scan_parquet(path_to_pc_df)
        pc_df = pc_df.rename(dict(zip(pc_df.collect_schema().names(), pc_cols)))
        logger.info("Data loaded successfully")

        # pipeline
        logger.info("Running pipeline ...")
        cleaned_df = (
            pp_df.with_columns(
                *clean_transform_postcode(postcode_re),
                clean_transform_price(price_re),
                clean_transform_date(dt_re),
                clean_category('property_type', ['F', 'S', 'D', 'T', 'O']),
                clean_category('newly_built', ['Y', 'N']),
                clean_category('freehold_or_lease', ['F', 'L'])
            )
            .filter(
                pl.col('date').is_between(start_date, end_date),
                pl.col('county').is_in(counties)
            )
            .drop('county')
            .unique(subset=['id'])
            .join(pc_df, on='postcode', how='left')
        )

    except Exception as e:
        logger.exception(f"Pipeline failed: {e}")
        raise
    finally:
        # Close handlers
        for handler in logger.handlers:
            handler.close()
            logger.removeHandler(handler)

    logger.info("Pipeline finished successfully")
    return cleaned_df.collect()

df_cleaned = pipeline()
df_cleaned

2025-09-19 09:47:27,406 - INFO - Loading data...
2025-09-19 09:47:27,495 - INFO - Data loaded successfully
2025-09-19 09:47:27,497 - INFO - Running pipeline ...
2025-09-19 09:47:27,535 - INFO - Pipeline finished successfully


id,price,date,postcode,property_type,newly_built,freehold_or_lease,paon,soan,street,locality,town_city,district,postal_sector,latitude,longitude
str,f64,datetime[μs],str,str,str,str,str,str,str,str,str,str,str,f64,f64
"""{01EB45EF-BCB5-40F3-E063-4704A…",225000.0,2023-04-24 00:00:00,"""OL9 0NG""","""T""","""N""","""L""","""955""","""""","""MIDDLETON ROAD""","""CHADDERTON""","""OLDHAM""","""OLDHAM""","""OL9 0""",53.548028,-2.157175
"""{3C17E818-0C2E-4827-9627-91F02…",42700.0,1999-07-02 00:00:00,"""M15 5AX""","""D""","""Y""","""L""","""107""","""""","""GREENHEYS LANE WEST""","""""","""MANCHESTER""","""MANCHESTER""","""M15 5""",53.462662,-2.249084
"""{DBA933F9-5C64-669D-E053-6B04A…",205000.0,2022-02-28 00:00:00,"""M24 4AR""","""S""","""N""","""L""","""39""","""""","""ASPELL CLOSE""","""MIDDLETON""","""MANCHESTER""","""ROCHDALE""","""M24 4""",53.550347,-2.204636
"""{5077A7AE-2769-49BC-A3D6-C8C63…",162000.0,2005-06-28 00:00:00,"""SK7 2DU""","""F""","""N""","""L""","""THE COPPICE""","""FLAT 14""","""BRAMHALL LANE SOUTH""","""BRAMHALL""","""STOCKPORT""","""STOCKPORT""","""SK7 2""",53.359961,-2.164897
"""{A0675032-3B95-4C0E-BCF3-C31B7…",32950.0,2004-12-03 00:00:00,"""WN7 2AQ""","""T""","""N""","""L""","""13""","""""","""LINGARD STREET""","""LEIGH""","""LEIGH""","""WIGAN""","""WN7 2""",53.494521,-2.50413
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""{7290EDFF-5EC4-470F-AA76-9222A…",330000.0,2013-06-06 00:00:00,"""M33 5QR""","""S""","""N""","""F""","""14""","""""","""DAVENHAM ROAD""","""""","""SALE""","""TRAFFORD""","""M33 5""",53.431167,-2.339717
"""{5A6B6DA1-1095-47F9-955B-88944…",52950.0,2001-05-10 00:00:00,"""M22 5FY""","""T""","""N""","""F""","""65""","""""","""CALVE CROFT ROAD""","""MANCHESTER""","""MANCHESTER""","""MANCHESTER""","""M22 5""",53.377243,-2.248976
"""{D5871689-422C-429F-85F5-68465…",70500.0,2007-11-16 00:00:00,"""WN7 4JB""","""T""","""N""","""F""","""53""","""""","""ETHERSTONE STREET""","""LEIGH""","""LEIGH""","""WIGAN""","""WN7 4""",53.492766,-2.523357


In [19]:
d = dt.now().strftime('%d-%m-%Y_%H-%M-%S')
df_cleaned.write_parquet(f'outputs\\paid_prices-{d}.parquet')